In [1]:

from pyspark.sql import SparkSession
import pyspark.sql.functions as f

if not 'spark' in locals():
    spark = SparkSession.builder \
         .master("spark://spark-master:7077")\
         .config("spark.executor.memory", "512m")\
        .getOrCreate()

spark

In [2]:
import pandas as pd

replication_df = spark.createDataFrame(pd.DataFrame(list(range(1,1000)),columns=['replication_id'])).repartition(1000, 'replication_id')

In [10]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


outSchema = StructType([StructField('replication_id', IntegerType(), True),
            StructField('sil_score', DoubleType(), True),
            StructField('num_clusters', IntegerType(), True),
            StructField('min_samples', IntegerType(), True),
            StructField('min_cluster_size', IntegerType(), True)])


@pandas_udf(outSchema, PandasUDFType.GROUPED_MAP)
def run_model(df_pandas):


    # Return result as a pandas data frame
    result = pd.DataFrame({'replication_id': replication_id, 'sil_score': 2,
                           'num_clusters': 3, 'min_samples': 4,
                           'min_cluster_size': 5}, index=[0])

    return result



results = replication_df.groupby("replication_id").apply(run_model)

In [11]:
spark.conf.set("spark.sql.pyspark.jvmStacktrace.enabled",True)
spark.conf.set("spark.sql.execution.pyspark.udf.simplifiedTraceback.enabled",False)

results.count()

PythonException: 
  An exception was thrown from Python worker in the executor. The below is the Python worker stacktrace.
Traceback (most recent call last):
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 589, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 421, in read_udfs
    arg_offsets, f = read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=0)
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 254, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 74, in read_command
    command = serializer._read_with_length(file)
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 172, in _read_with_length

In [12]:
spark.conf.set("spark.sql.pyspark.jvmStacktrace.enabled",False)
spark.conf.set("spark.sql.execution.pyspark.udf.simplifiedTraceback.enabled",False)

results.count()

PythonException: 
  An exception was thrown from Python worker in the executor. The below is the Python worker stacktrace.
Traceback (most recent call last):
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 589, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 421, in read_udfs
    arg_offsets, f = read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=0)
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 254, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 74, in read_command
    command = serializer._read_with_length(file)
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 172, in _read_with_length

In [13]:
spark.conf.set("spark.sql.pyspark.jvmStacktrace.enabled",False)
spark.conf.set("spark.sql.execution.pyspark.udf.simplifiedTraceback.enabled",True)

results.count()

PythonException: 
  An exception was thrown from Python worker in the executor. The below is the Python worker stacktrace.
Traceback (most recent call last):
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 589, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 421, in read_udfs
    arg_offsets, f = read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=0)
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 254, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 74, in read_command
    command = serializer._read_with_length(file)
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 172, in _read_with_length